In [ ]:
!pip install -r requirements.txt

In [1]:
# Import
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('../raw_data/export_2024-03-18.csv')
data.head()

,source,theme,soustheme,title,description,link,published,image,added_date
0,Écran large,culture,cinema,Test Final Fantasy 7 Rebirth : enfin le grand ...,Quatre ans après la sortie de Final Fantasy VI...,https://www.ecranlarge.com/jeux-video/critique...,"Sun, 17 Mar 2024 20:00:00 +0100",NaN,2024-03-18 09:00:00
1,Écran large,culture,cinema,True Detective : on a classé toutes les saison...,"La saison 1, la saison 2, la saison 3 ou la sa...",https://www.ecranlarge.com/series/dossier/1504...,"Sun, 17 Mar 2024 19:15:00 +0100",NaN,2024-03-18 09:00:00
2,Écran large,culture,cinema,Marvel change de stratégie et relance (encore)...,Le producteur Brad Winderbaum a annoncé que Ma...,https://www.ecranlarge.com/films/news/1505616-...,"Sun, 17 Mar 2024 17:00:00 +0100",NaN,2024-03-18 09:00:00
3,Écran large,culture,cinema,Dune 2 : le scénariste justifie cette énorme d...,"Jon Spaihts, le scénariste de Dune: Deuxième P...",https://www.ecranlarge.com/films/news/1505614-...,"Sun, 17 Mar 2024 16:00:00 +0100",NaN,2024-03-18 09:00:00
4,Écran large,culture,cinema,Dans la peau de Blanche Houellebecq : critique...,"Vendu comme une comédie grasse, Dans la peau d...",https://www.ecranlarge.com/films/critique/1505...,"Sun, 17 Mar 2024 14:45:00 +0100",NaN,2024-03-18 09:00:00


# Création de la TABLE Category

In [4]:
data.theme.unique()
theme_list = ['culture', 'musique', 'technologie', 'politique', 'sport', 'economie',
         'environnement', 'general', 'emploi', 'cite', 'management', 'presse',
         'nustrale','tourisme']
data = data[data.theme.isin(theme_list)]
data.theme.unique()

array(['culture', 'musique', 'technologie', 'politique', 'sport',
       'economie', 'environnement', 'general', 'emploi', 'cite',
       'management', 'presse', 'nustrale', 'tourisme'], dtype=object)

In [5]:
cat_df = pd.DataFrame({'category_id': list(range(1, len(theme_list) + 1)),
        'cat_name': theme_list})
cat_df

,category_id,cat_name
0,1,culture
1,2,musique
2,3,technologie
3,4,politique
4,5,sport
5,6,economie
6,7,environnement
7,8,general
8,9,emploi
9,10,cite


In [4]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

In [5]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
CAT_TABLE='category_dataset'
CAT_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{CAT_TABLE}' # the-mdr-project.live_mdr.category_dataset

In [13]:
client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
write_mode = 'WRITE_TRUNCATE'
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
job = client.load_table_from_dataframe(cat_df, CAT_TABLE_ID, job_config=job_config)
result = job.result()

## Test d'une query sur la category

In [14]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = "mathieu_credentials.json"

# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
CAT_TABLE='category_dataset'
CAT_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{CAT_TABLE}' # the-mdr-project.live_mdr.category_dataset

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
query = """SELECT *
            FROM the-mdr-project.live_mdr.category_dataset"""
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()
df

,category_id,cat_name
0,1,culture
1,2,musique
2,3,technologie
3,4,politique
4,5,sport
5,6,economie
6,7,environnement
7,8,general
8,9,emploi
9,10,cite


# Implémentation de la TABLE news

In [33]:
# Dataframe which is associate withe the news table
result = data.merge(cat_df, left_on='theme', right_on='cat_name')
result['news_id'] = np.arange(1, len(result) + 1)
news_df = result[['news_id', 'category_id', 'title', 'description', 'link', 'added_date', 'source', 'soustheme']]
news_df.rename(columns={'soustheme': 'sub_cat'}, inplace=True)
news_df

/var/folders/7_/qgxv4n054fz23g05r4yrv3kw0000gn/T/ipykernel_11701/3169457023.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.rename(columns={'soustheme': 'sub_cat'}, inplace=True)


,news_id,category_id,title,description,link,added_date,source,sub_cat
0,1,1,Test Final Fantasy 7 Rebirth : enfin le grand ...,Quatre ans après la sortie de Final Fantasy VI...,https://www.ecranlarge.com/jeux-video/critique...,2024-03-18 09:00:00,Écran large,cinema
1,2,1,True Detective : on a classé toutes les saison...,"La saison 1, la saison 2, la saison 3 ou la sa...",https://www.ecranlarge.com/series/dossier/1504...,2024-03-18 09:00:00,Écran large,cinema
2,3,1,Marvel change de stratégie et relance (encore)...,Le producteur Brad Winderbaum a annoncé que Ma...,https://www.ecranlarge.com/films/news/1505616-...,2024-03-18 09:00:00,Écran large,cinema
3,4,1,Dune 2 : le scénariste justifie cette énorme d...,"Jon Spaihts, le scénariste de Dune: Deuxième P...",https://www.ecranlarge.com/films/news/1505614-...,2024-03-18 09:00:00,Écran large,cinema
4,5,1,Dans la peau de Blanche Houellebecq : critique...,"Vendu comme une comédie grasse, Dans la peau d...",https://www.ecranlarge.com/films/critique/1505...,2024-03-18 09:00:00,Écran large,cinema
...,...,...,...,...,...,...,...,...
440580,440581,11,Comment mesurer le management ?,<p>Certains considèrent que le management cons...,https://www.hbrfrance.fr/management/comment-me...,2024-03-07 22:00:00,HBR,general
440581,440582,11,Les innovations managériales fondées sur le po...,<p>Alors que la troisième révolution industrie...,https://www.hbrfrance.fr/chroniques-experts/20...,2024-03-07 22:00:00,HBR,general
440582,440583,11,Penser le futur du travail implique aussi de p...,<p>L'IA et la crise écologique transforment le...,https://www.hbrfrance.fr/strategie/penser-le-f...,2024-03-07 22:00:00,HBR,general
440583,440584,11,Comment notre passion pour les réseaux sociaux...,<p>Dans un contexte où les utilisateurs produi...,https://www.hbrfrance.fr/magazine/2020/12/3231...,2024-03-07 22:00:00,HBR,general


In [34]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
NEWS_TABLE='news_dataset'
NEWS_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{NEWS_TABLE}' # the-mdr-project.live_mdr.news_dataset

# Path to your service account key file
key_path = "mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

In [35]:
client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
write_mode = 'WRITE_TRUNCATE'
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
job = client.load_table_from_dataframe(news_df, NEWS_TABLE_ID, job_config=job_config)
result = job.result()

## Test de l'implementation avec une query

In [36]:
query = """SELECT *
            FROM the-mdr-project.live_mdr.news_dataset
            LIMIT 10"""
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()
df

,news_id,category_id,title,description,link,added_date,source,sub_cat
0,29,1,"Arts vivants: Danse, performance: les corps s’...","Exploratoires, explosifs ou mélancoliques, les...",https://www.24heures.ch/danse-performance-les-...,2024-03-18 09:00:00,24heures : culture,None
1,30,1,Montreux avant le Jazz: Freddie et «Allo Claud...,Le déplacement temporaire de ces œuvres est di...,https://www.24heures.ch/montreux-jazz-festival...,2024-03-18 09:00:00,24heures : culture,None
2,31,1,Star de l’humour: Chantal Ladesou sera à Maxi-...,La comédienne a connu la célébrité sur le tard...,https://www.24heures.ch/humour-chantal-ladesou...,2024-03-18 09:00:00,24heures : culture,None
3,32,1,"Interview: Juliette, chanteuse: «On s’en fout ...","Truculente empêcheuse de penser en rond, la mu...",https://www.24heures.ch/juliette-chanteuse-on-...,2024-03-18 09:00:00,24heures : culture,None
4,97,1,Elon Musk vient à l’Élysée pour Bernard Arnaul...,"<p>Elon Musk, Beyoncé, Jay-Z ou encore Nicolas...",https://www.apar.tv/societe/elon-musk-vient-a-...,2024-03-18 09:00:00,Apar,None
5,98,1,"« Oui, je pense qu’il y a de la rancune » : Po...",<p>Le président russe Vladimir Poutine a souli...,https://www.apar.tv/societe/oui-je-pense-quil-...,2024-03-18 09:00:00,Apar,None
6,99,1,Gouel Sant Padrig mat deoc’h holl !,<p>Bonne Fête de la Saint Patrick ! Lá fhéile ...,https://www.argedour.bzh/gouel-sant-padrig-mat...,2024-03-18 09:00:00,Ar Gedour,None
7,103,1,Roman d’une Rencontre au musée de la Chaussure,Le musée de la Chaussure accueille Jean Paul G...,https://www.artsixmic.fr/2024-03-17-roman-dune...,2024-03-18 09:00:00,ArtSixMic : actualité,None
8,269,1,L'excellente série britannique It's a sin prop...,"À découvrir ce lundi 18 mars, tout au long de ...",https://www.leblogtvnews.com/2021/02/la-serie-...,2024-03-18 09:00:00,Blog TV news,None
9,270,1,La une de la presse quotidienne nationale ce l...,La une de la presse quotidienne nationale ce l...,https://www.leblogtvnews.com/2024/03/la-une-de...,2024-03-18 09:00:00,Blog TV news,None
